In [1]:
import os
import csv
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from math import radians, cos, sin, asin, sqrt
import gc
import warnings

warnings.filterwarnings('ignore')

In [2]:
path_train = "../resource/PINGAN-2018-train_demo.csv"
path_test = "../resource/PINGAN-2018-test_demo.csv"

In [3]:
all_feature_list = ['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1', 'dis', 'ave_dri_time', 'dri_time']

use_feature_list = [
    'trip_max', 'lon_max', 'lon_min', 'lon_50','lat_max', 'lat_min', 'lat_50', 'heg_max',
    'heg_min', 'heg_mean', 'heg_50', 'sp_max', 'sp_mean', 'sp_50', 'dis', 'avg_dis',
    'dri_time', 'ave_dri_time', 'dri_time_trip_max']


In [4]:
# 查看内存
keydic = {"MemTotal":"TotalMem","MemFree":"FreeMem","MemAvailable":"AvaiableMem","Cached":"Cached"}
def command(command):
    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    resultDic = {}
    for line in p.stdout.readlines():
        line = str(line,encoding="utf-8")
        result = re.split("\s*",line)
        if result[0][:-1] in keydic:
            resultDic[keydic[result[0][:-1]]] = "%.2f" %(int(result[1])/(1024**2))
    return resultDic

def load_data(path_train,path_test):
    train_data = pd.read_csv(path_train)
    test_data = pd.read_csv(path_test)
    return train_data,test_data


def read_csv():
    """
    文件读取模块，头文件见columns.
    :return: 
    """
    # for filename in os.listdir(path_train):
    tempdata = pd.read_csv(path_train)
    tempdata.columns = ["TERMINALNO", "TIME", "TRIP_ID", "LONGITUDE", "LATITUDE", "DIRECTION", "HEIGHT", "SPEED",
                        "CALLSTATE", "Y"]

#时间处理
def time_datetime(value):
    format = '%Y%m%d%H%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_date(value):
    format = '%Y%m%d'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_hour(value):
    format = '%H'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_minute(value):
    format = '%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)
#驾驶时长转换
def f(x):
    if x >= 20:
        return 0
    else:
        return x

In [5]:
# print(command("cat /proc/meminfo"))

In [6]:
# 总特征DF
feature = pd.DataFrame()

# 特征一,trip_max
train_data = pd.read_csv(path_train,usecols=['TERMINALNO', 'TIME', 'TRIP_ID'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'TIME', 'TRIP_ID'])

TRAIN_ID_MAX = train_data['TERMINALNO'].max() + 10
test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True, subset=['TERMINALNO','TIME'])

feature[['TERMINALNO', 'trip_max']] = data['TRIP_ID'].groupby(data['TERMINALNO']).max().reset_index()
del train_data, test_data, data
gc.collect()

82

In [7]:
# 特征二,lon_max lon_min lon_50
train_data = pd.read_csv(path_train,usecols=['TERMINALNO', 'LONGITUDE'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'LONGITUDE'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True)

feature[['TERMINALNO', 'lon_max']] = pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).max()).reset_index()
feature[['TERMINALNO', 'lon_min']] = pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).min()).reset_index()
feature[['TERMINALNO', 'lon_50']] = pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).quantile()).reset_index()
del train_data, test_data, data
gc.collect()

68

In [8]:
# 特征三,lat_max lat_min lat_50
train_data = pd.read_csv(path_train,usecols=['TERMINALNO', 'LATITUDE'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'LATITUDE'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True)

feature[['TERMINALNO', 'lat_max']] = pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).max()).reset_index()
feature[['TERMINALNO', 'lat_min']] = pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).min()).reset_index()
feature[['TERMINALNO', 'lat_50']] = pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).quantile()).reset_index()
del train_data, test_data, data
gc.collect()

89

In [9]:
# 特征四,heg_max heg_min heg_mean heg_50
train_data = pd.read_csv(path_train,usecols=['TERMINALNO', 'HEIGHT'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'HEIGHT'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True)
data.fillna(0.0, inplace=True)

feature[['TERMINALNO', 'heg_max']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).max()).reset_index()
feature[['TERMINALNO', 'heg_min']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).min()).reset_index()
feature[['TERMINALNO', 'heg_mean']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).mean()).reset_index()
feature[['TERMINALNO', 'heg_50']] = pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).quantile()).reset_index()
del train_data, test_data, data
gc.collect()

75

In [10]:
# 特征五,sp_max sp_mean sp_50
train_data = pd.read_csv(path_train,usecols=['TERMINALNO', 'SPEED'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'SPEED'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True)
data.fillna(0.0, inplace=True)

feature[['TERMINALNO', 'sp_max']] = pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).max()).reset_index()
feature[['TERMINALNO', 'sp_mean']] = pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).mean()).reset_index()
feature[['TERMINALNO', 'sp_50']] = pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).quantile()).reset_index()
del train_data, test_data, data
gc.collect()

68

In [11]:
# 特征七, 'dis', 'avg_dis'
train_data = pd.read_csv(path_train,usecols=['TERMINALNO','TIME', 'LONGITUDE', 'LATITUDE'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO','TIME', 'LONGITUDE', 'LATITUDE'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(inplace=True)
data.fillna(0.0, inplace=True)

# 每个用户按时间排序
data.sort_values(by=['TERMINALNO', 'TIME'], inplace=True)
# 计算经纬度差(未分Trip)
data['difflat'] = data.groupby(['TERMINALNO'])['LATITUDE'].diff()
data['difflon'] = data.groupby(['TERMINALNO'])['LONGITUDE'].diff()
# 对每个用户的第一个经纬度差置0
data.fillna(0.0, inplace=True)
# 计算单个距离
data['dis2'] = data['difflat'] ** 2 + data['difflon'] ** 2
data['dis'] = data['dis2'].apply(sqrt)
feature[['TERMINALNO', 'dis']] = pd.DataFrame(data['dis'].groupby(data['TERMINALNO']).sum()).reset_index()
feature['avg_dis'] = feature['dis'] / feature['trip_max']

del train_data, test_data, data
gc.collect()

96

In [12]:
# 特征八, dri_time ave_dri_time dri_time_trip_max
train_data = pd.read_csv(path_train, usecols=['TERMINALNO', 'TIME', 'TRIP_ID'])
test_data = pd.read_csv(path_test,usecols=['TERMINALNO', 'TIME', 'TRIP_ID'])

test_data['TERMINALNO'] = test_data['TERMINALNO'] + TRAIN_ID_MAX
data = pd.concat([train_data, test_data])
data.drop_duplicates(subset=['TERMINALNO', 'TIME'], inplace=True)
data.fillna(0.0, inplace=True)

# 按 TERMINALNO 和 time 排序
data.sort_values(['TERMINALNO', 'TIME'], inplace=True)
data['diff_time'] = data.groupby(['TERMINALNO'])['TIME'].diff()
data.fillna(0.0, inplace=True)
data['diff_time'] = data['diff_time'].apply(f)
dri_time_trip_max = pd.DataFrame()
dri_time_trip_max = pd.DataFrame(data.groupby(['TERMINALNO','TRIP_ID'])['diff_time'].sum()).reset_index()

# 计算驾驶总时长,用户单段最大驾驶时长(trip分割不准确)
feature[['TERMINALNO', 'dri_time']] = pd.DataFrame(data['diff_time'].groupby(data['TERMINALNO']).sum()).reset_index()
feature[['TERMINALNO', 'dri_time_trip_max']] = pd.DataFrame(dri_time_trip_max['diff_time'].groupby(dri_time_trip_max['TERMINALNO']).max()).reset_index()
feature['ave_dri_time'] = feature['dri_time'] / feature['trip_max']

del train_data, test_data, data, dri_time_trip_max
gc.collect()

150

In [13]:
# 归一化
feature['trip_max'] = feature['trip_max'].apply(
    lambda x: (x - feature['trip_max'].min()) / (feature['trip_max'].max() - feature['trip_max'].min()))
feature['lon_max'] = feature['lon_max'].apply(
    lambda x: (x - feature['lon_max'].min()) / (feature['lon_max'].max() - feature['lon_max'].min()))
feature['lon_min'] = feature['lon_min'].apply(
    lambda x: (x - feature['lon_min'].min()) / (feature['lon_min'].max() - feature['lon_min'].min()))
feature['lon_50'] = feature['lon_50'].apply(
    lambda x: (x - feature['lon_50'].min()) / (feature['lon_50'].max() - feature['lon_50'].min()))
feature['lat_min'] = feature['lat_min'].apply(
    lambda x: (x - feature['lat_min'].min()) / (feature['lat_min'].max() - feature['lat_min'].min()))
feature['lat_max'] = feature['lat_max'].apply(
    lambda x: (x - feature['lat_max'].min()) / (feature['lat_max'].max() - feature['lat_max'].min()))
feature['lat_50'] = feature['lat_50'].apply(
    lambda x: (x - feature['lat_50'].min()) / (feature['lat_50'].max() - feature['lat_50'].min()))
feature['heg_min'] = feature['heg_min'].apply(
    lambda x: (x - feature['heg_min'].min()) / (feature['heg_min'].max() - feature['heg_min'].min()))
feature['heg_max'] = feature['heg_max'].apply(
    lambda x: (x - feature['heg_max'].min()) / (feature['heg_max'].max() - feature['heg_max'].min()))
feature['heg_50'] = feature['heg_50'].apply(
    lambda x: (x - feature['heg_50'].min()) / (feature['heg_50'].max() - feature['heg_50'].min()))
feature['heg_mean'] = feature['heg_mean'].apply(
    lambda x: (x - feature['heg_mean'].min()) / (feature['heg_mean'].max() - feature['heg_mean'].min()))
feature['sp_50'] = feature['sp_50'].apply(
    lambda x: (x - feature['sp_50'].min()) / (feature['sp_50'].max() - feature['sp_50'].min()))
feature['sp_max'] = feature['sp_max'].apply(
    lambda x: (x - feature['sp_max'].min()) / (feature['sp_max'].max() - feature['sp_max'].min()))
feature['sp_mean'] = feature['sp_mean'].apply(
    lambda x: (x - feature['sp_mean'].min()) / (feature['sp_mean'].max() - feature['sp_mean'].min()))
feature['ave_dri_time'] = feature['ave_dri_time'].apply(
    lambda x: (x - feature['ave_dri_time'].min()) / (feature['ave_dri_time'].max() - feature['ave_dri_time'].min()))
feature['dri_time'] = feature['dri_time'].apply(
    lambda x: (x - feature['dri_time'].min()) / (feature['dri_time'].max() - feature['dri_time'].min()))
feature['dis'] = feature['dis'].apply(
    lambda x: (x - feature['dis'].min()) / (feature['dis'].max() - feature['dis'].min()))
feature['dri_time_trip_max'] = feature['dri_time_trip_max'].apply(
    lambda x: (x - feature['dri_time_trip_max'].min()) / (feature['dri_time_trip_max'].max() - feature['dri_time_trip_max'].min()))
feature['avg_dis'] = feature['avg_dis'].apply(
    lambda x: (x - feature['avg_dis'].min()) / (feature['avg_dis'].max() - feature['avg_dis'].min()))
print("data normalization..")
print("Feature End. feature shape:" + str(feature.shape))
print("generate train & test set")

data normalization..
Feature End. feature shape:(200, 20)
generate train & test set


In [14]:
 # train_Y
train_Y = pd.read_csv(path_train, usecols=['TERMINALNO', 'Y'])
train_Y.drop_duplicates(inplace=True)
# Y值变换
train_Y.loc[:, 'Y'][train_Y['Y'] <= 0] = 0.00001
import numpy as np
from scipy import stats
Y_arrary = np.array(train_Y['Y'])
y, _ = stats.boxcox(Y_arrary)
# for i in range(len(y)):
train_Y.loc[:, 'Y'] = y
del Y_arrary, y, _
gc.collect()

feature = pd.merge(feature, train_Y, how='left', on='TERMINALNO')
train = feature[0:len(train_Y)]
test = feature[len(train_Y):]
train['Y'] = train['Y'].apply(lambda x: ((x - train['Y'].min()) / (train['Y'].max() - train['Y'].min())))

In [15]:
# 训练集和验证集划分
train_train, train_val = train_test_split(train, test_size=0.2, random_state=42)
print("train_train_shape:"+str(train_train.shape)+"  train_val_shape:"+str(train_val.shape))
print("model training")

train_train_shape:(80, 21)  train_val_shape:(20, 21)
model training


In [18]:
#模型训练
lgbmodel = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='regression',
    num_leaves=63,
    max_depth=8,
    n_estimators=20000,
    learning_rate=0.05,
    # n_jobs=20,
    random_state=42
)
lgbmodel.fit(
    X=train_train[use_feature_list],
    y=train_train['Y'],
    eval_set=(train_val[use_feature_list], train_val['Y']),
    early_stopping_rounds=200,
    verbose=True
)

fea_imp = pd.Series(lgbmodel.feature_importances_,use_feature_list).sort_values(ascending=False)
print(fea_imp)

[1]	valid_0's l2: 0.127541
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 0.127185
[3]	valid_0's l2: 0.127148
[4]	valid_0's l2: 0.12698
[5]	valid_0's l2: 0.126757
[6]	valid_0's l2: 0.126655
[7]	valid_0's l2: 0.126502
[8]	valid_0's l2: 0.125333
[9]	valid_0's l2: 0.124278
[10]	valid_0's l2: 0.124151
[11]	valid_0's l2: 0.123211
[12]	valid_0's l2: 0.123114
[13]	valid_0's l2: 0.122278
[14]	valid_0's l2: 0.122205
[15]	valid_0's l2: 0.121064
[16]	valid_0's l2: 0.122048
[17]	valid_0's l2: 0.122226
[18]	valid_0's l2: 0.121563
[19]	valid_0's l2: 0.122814
[20]	valid_0's l2: 0.123806
[21]	valid_0's l2: 0.123189
[22]	valid_0's l2: 0.124417
[23]	valid_0's l2: 0.123478
[24]	valid_0's l2: 0.124485
[25]	valid_0's l2: 0.123832
[26]	valid_0's l2: 0.125031
[27]	valid_0's l2: 0.124484
[28]	valid_0's l2: 0.125486
[29]	valid_0's l2: 0.125283
[30]	valid_0's l2: 0.124784
[31]	valid_0's l2: 0.125967
[32]	valid_0's l2: 0.125499
[33]	valid_0's l2: 0.126508
[34]	valid_0's l2: 0.12